In [1]:
%matplotlib inline
import numpy as np
import cv2
import os, time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from helper import *

In [2]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 1 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (8, 8) # Spatial binning dimensions
hist_bins = 8    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

In [3]:
vehicle_path = "./dataset/OwnCollection/vehicles/"
nonvehicle_path = "./dataset/OwnCollection/non-vehicles/"

vehicle_list = get_dataset(vehicle_path)
nonvehicle_list = get_dataset(nonvehicle_path)

In [4]:
## need to test SVC acc
## color space: RGB, LUV, HLS, YCrCb
## pix_per_cell: 8, 16
## cell_per_block: 1, 2
## others fixed

color_list = ['RGB', 'LUV','HLS','YCrCb']
orient_list = [9, 12]
pix_per_cell_list = [8, 16]
cell_per_block_list = [1, 2]
parameter_set = []

for cs in color_list:
    for ppc in pix_per_cell_list:
        for cpb in cell_per_block_list:
            for orient in orient_list:
                parameter_set.append([cs, ppc, cpb, orient])

parameter_set

[['RGB', 8, 1, 9],
 ['RGB', 8, 1, 12],
 ['RGB', 8, 2, 9],
 ['RGB', 8, 2, 12],
 ['RGB', 16, 1, 9],
 ['RGB', 16, 1, 12],
 ['RGB', 16, 2, 9],
 ['RGB', 16, 2, 12],
 ['LUV', 8, 1, 9],
 ['LUV', 8, 1, 12],
 ['LUV', 8, 2, 9],
 ['LUV', 8, 2, 12],
 ['LUV', 16, 1, 9],
 ['LUV', 16, 1, 12],
 ['LUV', 16, 2, 9],
 ['LUV', 16, 2, 12],
 ['HLS', 8, 1, 9],
 ['HLS', 8, 1, 12],
 ['HLS', 8, 2, 9],
 ['HLS', 8, 2, 12],
 ['HLS', 16, 1, 9],
 ['HLS', 16, 1, 12],
 ['HLS', 16, 2, 9],
 ['HLS', 16, 2, 12],
 ['YCrCb', 8, 1, 9],
 ['YCrCb', 8, 1, 12],
 ['YCrCb', 8, 2, 9],
 ['YCrCb', 8, 2, 12],
 ['YCrCb', 16, 1, 9],
 ['YCrCb', 16, 1, 12],
 ['YCrCb', 16, 2, 9],
 ['YCrCb', 16, 2, 12]]

In [5]:
def get_classifier(vehicle_list, nonvehicle_list, parameters):

    color_space = parameters[0] # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = parameters[3]  # HOG orientations
    pix_per_cell = parameters[1] # HOG pixels per cell
    cell_per_block = parameters[2] # HOG cells per block
    hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    spatial_size = (32, 32) # Spatial binning dimensions
    hist_bins = 32    # Number of histogram bins
    spatial_feat = True # Spatial features on or off
    hist_feat = True # Histogram features on or off
    hog_feat = True # HOG features on or off
    
    car_features = extract_features(vehicle_list, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
    notcar_features = extract_features(nonvehicle_list, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)

    # Standardized dataset
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    X_scaler = StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)

    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=101)

    # Train SVM classifier
    svc = LinearSVC()

    t=time.time()
    svc.fit(X_train, y_train)
    t2=time.time()
    print(parameters)
    print(round(t2-t, 2), 'Seconds to train SVC...')
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    print('\n')

In [6]:
for parameter in parameter_set:
    get_classifier(vehicle_list=vehicle_list, nonvehicle_list=nonvehicle_list, parameters=parameter)

/home/vmadmin/.local/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


['RGB', 8, 1, 9]
16.96 Seconds to train SVC...
Test Accuracy of SVC =  0.9845


['RGB', 8, 1, 12]
20.97 Seconds to train SVC...
Test Accuracy of SVC =  0.9834


['RGB', 8, 2, 9]
32.79 Seconds to train SVC...
Test Accuracy of SVC =  0.9862


['RGB', 8, 2, 12]
41.76 Seconds to train SVC...
Test Accuracy of SVC =  0.9831


['RGB', 16, 1, 9]
11.83 Seconds to train SVC...
Test Accuracy of SVC =  0.9814


['RGB', 16, 1, 12]
11.24 Seconds to train SVC...
Test Accuracy of SVC =  0.9789


['RGB', 16, 2, 9]
13.4 Seconds to train SVC...
Test Accuracy of SVC =  0.9792


['RGB', 16, 2, 12]
14.57 Seconds to train SVC...
Test Accuracy of SVC =  0.9851


['LUV', 8, 1, 9]
16.17 Seconds to train SVC...
Test Accuracy of SVC =  0.991


['LUV', 8, 1, 12]
19.68 Seconds to train SVC...
Test Accuracy of SVC =  0.993


['LUV', 8, 2, 9]
30.07 Seconds to train SVC...
Test Accuracy of SVC =  0.9924


['LUV', 8, 2, 12]
40.17 Seconds to train SVC...
Test Accuracy of SVC =  0.9904


['LUV', 16, 1, 9]
9.79 Seconds to